# Import new noiseLAB data into main dataset

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from nptdms import TdmsFile
import imp
import seaborn as sns

from aeroacoustics import dataset as ds
from aeroacoustics import read_data_files as rd

In [2]:
imp.reload(rd)
imp.reload(ds)

<module 'aeroacoustics.dataset' from '/Users/esimley/Documents/Code/Aeroacoustics/coderepo/aeroacoustics/dataset.py'>

In [3]:
%matplotlib

Using matplotlib backend: MacOSX


## Parameters

In [4]:
# root_dir_noiselab='/Volumes/Aeroacoustics/_ProcessedData/SliceData'
root_dir_noiselab='/Volumes/Aeroacoustics/NoiseLAB/DOE15AeroacousticsProject/Results/Tables'
root_dir_tdms='/Volumes/Tests/_raw data/Slow'

# list_filename='noiseLAB_files.p'
# dataset_file='aeroacoustics_dataset.p'

list_filename='/Volumes/Aeroacoustics/_ProcessedData/Dataset/noiseLAB_files.p'
list_filename_unweighted='/Volumes/Aeroacoustics/_ProcessedData/Dataset/noiseLAB_files_unweighted.p'
dataset_file='/Volumes/Aeroacoustics/_ProcessedData/Dataset/aeroacoustics_dataset.p'
dataset_file_unweighted='/Volumes/Aeroacoustics/_ProcessedData/Dataset/aeroacoustics_dataset_unweighted.p'
# dataset_file='aeroacoustics_dataset.p'

list_filename_fft='/Volumes/Aeroacoustics/_ProcessedData/Dataset/noiseLAB_files_fft.p'
list_filename_fft_unweighted='/Volumes/Aeroacoustics/_ProcessedData/Dataset/noiseLAB_files_fft_unweighted.p'
dataset_file_fft='/Volumes/Aeroacoustics/_ProcessedData/Dataset/aeroacoustics_dataset_fft.p'
dataset_file_fft_unweighted='/Volumes/Aeroacoustics/_ProcessedData/Dataset/aeroacoustics_dataset_fft_unweighted.p'
# dataset_file_fft='aeroacoustics_dataset_fft.p'

fig_dir = '/Volumes/Aeroacoustics/_ProcessedData/Figures'

## Load data

In [32]:
df_unw = pd.read_pickle(dataset_file_unweighted)
df_Awt = pd.read_pickle(dataset_file)


In [33]:
print(len(df_unw))
print(len(df_Awt))

5447
5511


In [34]:
plt.figure()
# plt.plot(df_unw['EGD_OpCtl_TurbineStatus'])
# plt.plot(np.isnan(df_unw.WD_Nacelle_Mod))
# plt.plot(np.isnan(df_Awt.WD_Nacelle_Mod))
plt.plot(df_filt['Yaw_Position_Normal'])

In [7]:
# define 1/3 octave band center frequencies
third_oct_center_freqs_unw = np.array([1, 1.2, 1.6, 2, 2.5, 3.2, 4, 5, 6.3, 8, 10, 12.5, 16, 20, 25, 31.5, 40, 50, 63, 80, 100, 125, 160, 200, 250, 315, 400, 500, 630, 800, 1000, 1250, 1600, 2000, 2500, 3150, 4000, 5000, 6300, 8000, 10000, 12500, 16000, 20000])
third_oct_center_freqs_Awt = np.array([20, 25, 31.5, 40, 50, 63, 80, 100, 125, 160, 200, 250, 315, 400, 500, 630, 800, 1000, 1250, 1600, 2000, 2500, 3150, 4000, 5000, 6300, 8000, 10000, 12500, 16000, 20000])

### Filter data

In [35]:
# use_Awt = True
use_Awt = False

if use_Awt:
    df_filt = df_Awt.copy()
else:
    df_filt = df_unw.copy()

print(len(df_filt))

# TEMP!
# df_filt['Yaw_Offset_Cmd'] = df_filt.WD_Nacelle - df_filt.WD_Nacelle_Mod

# df_filt['Yaw_Offset_Cmd_Bin'] = 0.
# df_filt.loc[df_filt['Yaw_Offset_Cmd'] < -9.,'Yaw_Offset_Cmd_Bin'] = -18.
# df_filt.loc[(df_filt['Yaw_Offset_Cmd'] >= 5.) & (df_filt['Yaw_Offset_Cmd'] < 14.),'Yaw_Offset_Cmd_Bin'] = 10.
# df_filt.loc[(df_filt['Yaw_Offset_Cmd'] >= 14.) & (df_filt['Yaw_Offset_Cmd'] < 21.5),'Yaw_Offset_Cmd_Bin'] = 18.
# df_filt.loc[df_filt['Yaw_Offset_Cmd'] >= 21.5,'Yaw_Offset_Cmd_Bin'] = 25.

# plt.figure()
# plt.plot(df_filt['Active Power'])

# get rid of spurious offset commands
# df_filt.loc[df_filt.index < '2021-01-01','Yaw_Offset_Cmd_Bin'] = 0.

# filter to remove 5 minutes after a yaw offset command change
change_points = df_filt.loc[df_filt['Yaw_Offset_Cmd_Bin'].diff().abs() > 0].index
for d in change_points:
    df_filt = df_filt[(df_filt.index < d) | (df_filt.index >= d + pd.Timedelta('5 min'))]

print(len(df_filt))
    
# create derived signals
df_filt['Turbine Operating'] = (df_filt['EGD_OpCtl_TurbineStatus'] == 2)

# catch operation based on power too, and remove from dataset
df_filt = df_filt[(df_filt['Active Power'] < 10) | df_filt['Turbine Operating']]
print(len(df_filt))
# This should catch cases where the yaw offset command is NaN beause the turbine is not running
df_filt.loc[~df_filt['Turbine Operating'],'Yaw_Offset_Cmd_Bin'] = 0.
df_filt['Yaw_Position_Normal'] = (df_filt['Yaw_Encoder'] + df_filt['Yaw_Offset_Cmd_Bin']) % 360.

# plt.plot(df_filt['Active Power'])

# filter to yaw position +/- 15 degrees from 285
df_filt['Direction Valid'] = (df_filt['Yaw_Position_Normal'] >= 270.) & (df_filt['Yaw_Position_Normal'] <= 300.)
df_filt = df_filt[df_filt['Direction Valid']]
print(len(df_filt))
# plt.plot(df_filt['Active Power'],'r')

# filter to rows when all mics are available
cols2check = ['01N_Leq', '02N_Leq', '03N_Leq', '04C_Leq', '05C_Leq', '06S_Leq', '07S_Leq', '08S_Leq', '11N_Leq', '12C_Leq', '13S_Leq']
df_filt = df_filt.dropna(subset=cols2check)
print(len(df_filt))

5447
4646
4081
1595
1587


In [36]:
plt.figure()
plt.plot(df_filt['Temp1'])

## Plot 1/3 octave band spectra by wind speed for a given mic and offset

In [10]:
def energy_avg(values):
    return 10*np.log10(np.mean(10**(values/10)))

def energy_subtract(values1,values2):
    return 10*np.log10(10**(values1/10) - 10**(values2/10))

In [18]:
df_filt.groupby(ws_col).mean().columns.values

array(['01N_Leq', '02N_Leq', '03N_Leq', '04C_Leq', '05C_Leq', '06S_Leq',
       '07S_Leq', '08S_Leq', '11N_Leq', '12C_Leq', '13S_Leq',
       '01N_G (infra)', '02N_G (infra)', '03N_G (infra)', '04C_G (infra)',
       '05C_G (infra)', '06S_G (infra)', '07S_G (infra)', '08S_G (infra)',
       '11N_G (infra)', '12C_G (infra)', '13S_G (infra)', '01N_A (LF)',
       '02N_A (LF)', '03N_A (LF)', '04C_A (LF)', '05C_A (LF)',
       '06S_A (LF)', '07S_A (LF)', '08S_A (LF)', '11N_A (LF)',
       '12C_A (LF)', '13S_A (LF)', '01N_Lin', '02N_Lin', '03N_Lin',
       '04C_Lin', '05C_Lin', '06S_Lin', '07S_Lin', '08S_Lin', '11N_Lin',
       '12C_Lin', '13S_Lin', 'MS Excel Timestamp', 'LabVIEW Timestamp',
       'Scan Errors', 'Late Scans', 'SCADA_nacelleWS', 'Hum1', 'Temp1',
       'Hum2', 'Temp2', 'Windspeed_38m', 'Windspeed_55m', 'Windspeed_87m',
       'WD1_87m', 'Air_Press_2', 'Air_Press_1', 'Wind_Direction_38m',
       'WindSpeed_80m', 'Precipitation', 'Tower Base Bend 1',
       'Tower Base Bend 2

In [ ]:
# plt.figure()
# plt.scatter(df_filt.loc[~df_filt['Turbine Operating'],ws_col],df_filt.loc[~df_filt['Turbine Operating'],'Active Power'])

In [38]:
ws_col = 'WindSpeed_80m_round'

yaw_offset_targets = [0.,-18.,10.,18.,25.]

df_filt['WindSpeed_80m_round'] = df_filt['WindSpeed_80m'].round()

yaw_offset_target = 10.

mic = '12C'

ws_list = np.arange(8.,17.,1.)

if use_Awt:
    center_freqs = third_oct_center_freqs_Awt
else:
    center_freqs = third_oct_center_freqs_unw

# first plot Leq vs wind speed
fig = plt.figure()

Leq_operating = np.zeros(len(ws_list))
Leq_background = np.zeros(len(ws_list))

for i_ws, ws in enumerate(ws_list):
    Leq_operating[i_ws] = energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_Leq'])
    Leq_background[i_ws] = energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & ~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_Leq'])

plt.plot(ws_list,Leq_operating,color='g',linewidth=2.,label='Operating')
plt.plot(ws_list,Leq_background,color='b',linewidth=2.,label='Background')
plt.plot(ws_list,energy_subtract(Leq_operating,Leq_background),color='r',linewidth=2.,label='Corrected')

plt.xlabel('Wind Speed (m/s)')

if use_Awt:
    plt.ylabel('$L_{eq}$ (dBA)')
else:
    plt.ylabel('$L_{eq}$ (dB)')
    
plt.grid()
plt.title('Mic '+mic+', Yaw Offset = '+str(yaw_offset_target)+'$^\circ$')
plt.legend()
plt.xlim([ws_list[0],ws_list[-1]])
    
if use_Awt:
    fig.savefig(os.path.join(fig_dir,'LAeq_'+mic+'_'+str(yaw_offset_target)+'degOffset.pdf'), bbox_inches='tight')
else:
    fig.savefig(os.path.join(fig_dir,'Leq_'+mic+'_'+str(yaw_offset_target)+'degOffset.pdf'), bbox_inches='tight')
    
fig, axs = plt.subplots(3,4,sharex=True,sharey=True,figsize=(14,9))

ws_list = np.arange(8.,17.,1.)

for i_ws, ws in enumerate(ws_list):

    Leq_operating = energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_third_octave_spectrum'])
    Leq_background = energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & ~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_third_octave_spectrum'])
    
    axs[i_ws % 3][int(np.floor(i_ws/3))].plot(center_freqs,Leq_operating,color='g',linewidth=1.5,label='Operating')
    if len(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & ~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_third_octave_spectrum']) > 0:
        axs[i_ws % 3][int(np.floor(i_ws/3))].plot(center_freqs,Leq_background,color='b',linewidth=1.5,label='Background')
        axs[i_ws % 3][int(np.floor(i_ws/3))].plot(center_freqs,energy_subtract(Leq_operating,Leq_background),color='r',linewidth=1.5,label='Corrected')
    axs[i_ws % 3][int(np.floor(i_ws/3))].legend()
    axs[i_ws % 3][int(np.floor(i_ws/3))].grid()
    axs[i_ws % 3][int(np.floor(i_ws/3))].set_title('Mic '+mic+', Yaw Offset = '+str(yaw_offset_target)+'$^\circ$, '+str(ws)+' m/s')
    axs[i_ws % 3][int(np.floor(i_ws/3))].set_xscale('log')
    
for i in range(3):
    if use_Awt:
        axs[i][0].set_ylabel('$L_{eq}$ (dBA)')
    else:
        axs[i][0].set_ylabel('$L_{eq}$ (dB)')
        
for i in range(4):
    axs[2][i].set_xlabel('Frequency (Hz)')

if use_Awt:
    axs[0][0].set_xlim([20,10000])
else:
    axs[0][0].set_xlim([1,10000])
    
plt.tight_layout()

if use_Awt:
    fig.savefig(os.path.join(fig_dir,'ThirdOctave_LAeq_'+mic+'_'+str(yaw_offset_target)+'degOffset.pdf'), bbox_inches='tight')
else:
    fig.savefig(os.path.join(fig_dir,'ThirdOctave_Leq_'+mic+'_'+str(yaw_offset_target)+'degOffset.pdf'), bbox_inches='tight')

In [24]:
# ~np.isnan(Leq_background)
len(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & ~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_third_octave_spectrum'])

0

In [108]:
energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_third_octave_spectrum'])

array([-2.84104838, -2.52899151,  2.71574782,  4.16608488,  8.81127343,
       15.04275796, 15.05423191, 16.71510428, 16.0905686 , 17.87380912,
       17.42583545, 18.61400097, 19.81887721, 21.12404309, 22.90012419,
       23.49495368, 26.65405244, 26.8898906 , 27.2484863 , 26.33432968,
       25.94164944, 26.39567744, 27.03224495, 27.70734759, 28.24681193,
       28.54788246, 28.51960987, 28.05971642, 27.09254795, 25.63379752,
       24.0525524 ])

In [140]:
plt.figure()
plt.scatter(df_filt.loc[~df_filt['Turbine Operating'],'NacelleWindSpeed'],df_filt.loc[~df_filt['Turbine Operating'],'Active Power'])
plt.scatter(df_filt.loc[~df_filt['Turbine Operating'],ws_col],df_filt.loc[~df_filt['Turbine Operating'],'Active Power'])
# plt.scatter(df_filt.loc[~df_filt['Turbine Operating'],'NacelleWindSpeed'],df_filt.loc[~df_filt['Turbine Operating'],'LSS RPM'])
# plt.scatter(df_filt.loc[~df_filt['Turbine Operating'],ws_col],df_filt.loc[~df_filt['Turbine Operating'],'LSS RPM'])


In [150]:
plt.figure()
# plt.plot(df['Yaw_Offset_Cmd_Bin'])
# plt.plot(df[ws_col])
for k in [3,4,9]:
    plt.plot(df_filt[cols2check[k]])
# plt.plot(df['Active Power'])
plt.plot(df_filt['Yaw_Offset_Cmd_Bin'])
# plt.plot(df['Yaw_Offset_Cmd'])
# plt.plot(df['WD_Nacelle'])
# plt.plot(df['Yaw_Encoder'],label='Yaw Encoder')
# plt.plot(df['Yaw_Position_Normal'],label='Yaw_Position_Normal')
# # plt.plot(df['WD1_87m'],label='WD1_87m')
# plt.plot(df['Wind_Direction_38m'],label='Wind_Direction_38m')
# plt.legend()